In [1]:
# import libraries
import datetime
from datetime import datetime
from pandas_datareader import data as pdr
import yfinance as yf
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import random
import warnings
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_csv('exports/stocks.csv')
df

,stock_name,stock_code
0,7-ELEVEN MALAYSIA HOLDINGS BERHAD,5250
1,ABF MALAYSIA BOND INDEX FUND,0800EA
2,ABLE GLOBAL BERHAD,7167
3,ABLEGROUP BERHAD,7086
4,ABM FUJIYA BERHAD,5198
5,ACE INNOVATE ASIA BERHAD,03028
6,ACME HOLDINGS BERHAD,7131
7,ACO GROUP BERHAD,0218
8,ADVANCE INFORMATION MARKETING BERHAD,0122
9,ADVANCE SYNERGY BERHAD,1481


In [3]:
# Add ".KL" to each value in the 'stock_code' column
testdf = df['stock_code'].apply(lambda x: x + ".KL")
testdf

0         5250.KL
1       0800EA.KL
2         7167.KL
3         7086.KL
4         5198.KL
5        03028.KL
6         7131.KL
7         0218.KL
8         0122.KL
9         1481.KL
10        5281.KL
11        9148.KL
12        7191.KL
13        7146.KL
14        0181.KL
15        6599.KL
16        5139.KL
17        5185.KL
18        7145.KL
19        0258.KL
20        7315.KL
21        7078.KL
22        0209.KL
23        5238.KL
24        2658.KL
25        7609.KL
26        5116.KL
27        5115.KL
28        2674.KL
29        0079.KL
30        2488.KL
31        1163.KL
32       03051.KL
33        5269.KL
34        5127.KL
35        5293.KL
36        5307.KL
37        5120.KL
38       03011.KL
39        1015.KL
40        7031.KL
41        6351.KL
42        7083.KL
43        0048.KL
44        4758.KL
45        0226.KL
46        6556.KL
47        5082.KL
48        5568.KL
49        5088.KL
50        7090.KL
51        5015.KL
52        6432.KL
53        0119.KL
54        7214.KL
55        

In [4]:
# Put it into a list called stock_code
stock_code = list(testdf)[:5]
stock_code

['5250.KL', '0800EA.KL', '7167.KL', '7086.KL', '5198.KL']

In [11]:
def convert_date_string_to_datetime(date_str):
    # Define the format of the input date string
    date_format = "%d %b %Y"  # For example: "31 MAR 2023"

    try:
        # Parse the date string into a datetime object
        date_obj = datetime.strptime(date_str, date_format)
        return date_obj
    except ValueError:
        # Handle the case where the date string is not in the expected format
        return None  # You can choose to return None or raise an exception

In [16]:
driver = webdriver.Chrome()
driver.implicitly_wait(20)

# funtion to get link
def get_link(stock_code):
    
    url='https://www.bursamarketplace.com/index.php'
    driver.get(url)
    
    # Clicking the Search Button
    s= driver.find_element(By.XPATH,'//*[@id="newnav-mobileSearch"]')
    driver.execute_script("arguments[0].click();",s)
    
    # Inputting the Stock Code in the Search Bar
    driver.find_element(By.XPATH, '//*[@id="newnav-search-input"]').send_keys(stock_code)
    
    # Getting the Link in the appeared Search Results
    lnks = driver.find_element(By.ID,'url')
    links=lnks.get_attribute('href')
    return links

In [17]:
results_df = []
new_list = []

# Define the columns you want to retrieve
columns_to_fetch = ["dividendYield", "payoutRatio", "operatingMargins"]

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=["Date", "Revenue"])

# Loop through each ticker in the CSV
for ticker_symbol in stock_code:
    try:
        url=get_link(ticker_symbol[:-3]) # Exclude ".KL for Bursa Marketplace Search Function"
        url = url + '/financials_IS_Qt'  # Append the desired subpage to the URL
        driver.get(url)
        content = driver.page_source
        soup = bs(content)

        time.sleep(2)

        # Initialize the row dictionary to store the data
        row = {}

        # Find the element with the text 'MYR (MILLION)'
        myr_element = driver.find_element(By.XPATH, "//div[text()='MYR (MILLION)']")

        # Find the first sibling of the 'MYR (MILLION)' element
        first_sibling = myr_element.find_element(By.XPATH, "following-sibling::div")

        # Find the element with a class value of 'tb_cell tb_metr' and contains the text '&nbsp;'
        first_sibling_year = driver.find_element(By.XPATH, "//div[contains(@class, 'tb_cell tb_metr')]")

        # Find the element with the "Revenue" span text
        revenue_prev_element = driver.find_element(By.XPATH, "//div[span='Revenue']")

        # Iterate through the next 4 siblings to get the latest 4 quarter dates
        quarter_dates = []

        for i in range(4):
            quarter_date = first_sibling.find_element(By.XPATH, "following-sibling::div")
            year = first_sibling_year.find_element(By.XPATH, "following-sibling::div")
            if not year.text.strip():
                if previous_year:
                    current_date = quarter_date.text + " " + previous_year.text
            else:
                current_date = quarter_date.text + " " + year.text
                previous_year = year  # Update the previous_year

            date_obj = convert_date_string_to_datetime(current_date)

            revenue_element = revenue_prev_element.find_element(By.XPATH, "following-sibling::div")

            row["Date"] = date_obj
            row["Revenue"] = revenue_element.text

            # Append the row to the results DataFrame
            results_df = results_df.append(row, ignore_index=True)

            first_sibling_year = year
            first_sibling = quarter_date
            revenue_prev_element = revenue_element
            
    except Exception as e:
        print(f"Error fetching data for {ticker_symbol}: {str(e)}")
        new_list.append(ticker_symbol)
  

print(results_df)
    




C:\Users\Aidan\AppData\Local\Temp\ipykernel_3208\1214120213.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_3208\1214120213.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_3208\1214120213.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_3208\1214120213.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.app

Error fetching data for 0800EA.KL: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[text()='MYR (MILLION)']"}
  (Session info: chrome=118.0.5993.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6A9F08EF2+54786]
	(No symbol) [0x00007FF6A9E75612]
	(No symbol) [0x00007FF6A9D2A64B]
	(No symbol) [0x00007FF6A9D6B79C]
	(No symbol) [0x00007FF6A9D6B91C]
	(No symbol) [0x00007FF6A9DA6D87]
	(No symbol) [0x00007FF6A9D8BEAF]
	(No symbol) [0x00007FF6A9DA4D02]
	(No symbol) [0x00007FF6A9D8BC43]
	(No symbol) [0x00007FF6A9D60941]
	(No symbol) [0x00007FF6A9D61B84]
	GetHandleVerifier [0x00007FF6AA257F52+3524194]
	GetHandleVerifier [0x00007FF6AA2AD800+3874576]
	GetHandleVerifier [0x00007FF6AA2A5D7F+3843215]
	GetHandleVerifier [0x00007FF6A9FA5086+694166]
	(No symbol) [0x00007FF6A9E80A88]
	(No symbol) [0x00007FF6A9E7CA94]
	(No symb

C:\Users\Aidan\AppData\Local\Temp\ipykernel_3208\1214120213.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_3208\1214120213.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_3208\1214120213.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_3208\1214120213.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.app

           Date   Revenue
0   31 MAR 2023  1,066.45
1   31 DEC 2023    976.92
2   30 SEP 2022    992.42
3   30 JUN 2022    988.21
4   31 MAR 2023    156.28
5   31 DEC 2023    146.13
6   30 SEP 2022    127.77
7   30 JUN 2022    139.14
8   31 MAR 2023      1.41
9   31 DEC 2023      1.45
10  30 SEP 2022      1.57
11  30 JUN 2022      1.18
12  31 MAR 2023     33.24
13  31 DEC 2023     25.39
14  30 SEP 2022     26.38
15  30 JUN 2022     25.23


C:\Users\Aidan\AppData\Local\Temp\ipykernel_3208\1214120213.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_3208\1214120213.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_3208\1214120213.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append(row, ignore_index=True)
C:\Users\Aidan\AppData\Local\Temp\ipykernel_3208\1214120213.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.app